In [1]:
import torch
import math
import numpy as np
import json

In [2]:
from lyft_simulation import *

In [5]:
num_drivers = 100
num_subgrids = 16

In [12]:
num_subgrids_per_dim = 4
num_subgrids = num_subgrids_per_dim ** 2

In [13]:
subgrid_size = 1.0 / num_subgrids_per_dim
subgrid_bounds = torch.zeros(num_subgrids, 2, 2)
for i in range(num_subgrids_per_dim):
    for j in range(num_subgrids_per_dim):
        subgrid_index = i * num_subgrids_per_dim + j
        subgrid_bounds[subgrid_index, 0] = torch.tensor([i * subgrid_size, (i + 1) * subgrid_size])
        subgrid_bounds[subgrid_index, 1] = torch.tensor([j * subgrid_size, (j + 1) * subgrid_size])


In [15]:
subgrid_bounds.size()

torch.Size([16, 2, 2])

In [28]:
# Assign each driver to a subgrid
driver_subgrids = torch.randint(0, num_subgrids, (num_drivers,))

# Initialize the driver positions randomly within their assigned subgrids
driver_positions = torch.zeros(num_drivers, 2)

In [31]:
for i in range(num_subgrids):
    mask = driver_subgrids == i
    x_min, x_max = subgrid_bounds[i, 0]
    y_min, y_max = subgrid_bounds[i, 1]
    driver_positions[mask, 0] = torch.rand(mask.sum()) * (x_max - x_min) + x_min
    driver_positions[mask, 1] = torch.rand(mask.sum()) * (y_max - y_min) + y_min

In [35]:
# Define the probability distribution for driver idle times
daytime_prob = 0.8  # Probability of drivers working during daytime hours
nighttime_prob = 1 - daytime_prob
daytime_hours = torch.arange(7, 23)  # Daytime hours (7 AM to 10 PM)
nighttime_hours = torch.cat((torch.arange(0, 7), torch.arange(23, 24)))  # Nighttime hours (12 AM to 7 AM and 11 PM to 12 AM)
idle_time_probs = torch.zeros(24)
idle_time_probs[daytime_hours] = daytime_prob
idle_time_probs[nighttime_hours] = nighttime_prob
idle_time_probs /= idle_time_probs.sum()

In [36]:
idle_time_probs

tensor([0.0139, 0.0139, 0.0139, 0.0139, 0.0139, 0.0139, 0.0139, 0.0556, 0.0556,
        0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0556,
        0.0556, 0.0556, 0.0556, 0.0556, 0.0556, 0.0139])

In [37]:
idle_starttime = torch.multinomial(idle_time_probs, num_drivers, replacement=True) * 60

In [38]:
idle_starttime

tensor([1380,  780,  600,  960,  720,  600,  420,  780, 1080, 1260,  960,  900,
        1080, 1320, 1200, 1260,  840, 1320, 1020, 1380,  420,  900, 1200,  600,
         840,  840, 1260, 1200,  540,  660, 1080, 1200, 1320, 1020,  960, 1380,
         480,  420,  420,  480, 1020,  540,  960, 1320, 1260,  840,  240, 1260,
        1140, 1140, 1260, 1140,  600,  720,  720, 1140,  660, 1260, 1320,  240,
         480, 1020, 1020,  540, 1320,  420, 1200, 1320,  600,  960, 1020,  840,
        1020,  720,  720,  540, 1200, 1320,  960,  540, 1140, 1200, 1080,  720,
         660, 1260,  780,  660,  540,  840,  840,  360,  480,  660,  840,  420,
        1140,    0,  660,  660])

In [24]:
# on avg. a driver can work for 3hrs
mean_idle_time = 180

In [25]:
exponential_dist = torch.distributions.exponential.Exponential(torch.tensor(1.0 / mean_idle_time))

In [26]:
idle_duration = exponential_dist.sample((num_drivers,)).clamp(min=1, max=24 * 60).int().long()

In [27]:
idle_duration

tensor([ 28, 371,   5, 113,  25,  14,  20, 112, 130, 768, 150, 261, 561,  84,
         14,  66,  58, 113,  42,   3,   6,   3, 179, 666,  71, 294,  43,  31,
        127, 113, 517, 198, 123, 122,   7,  30,  34, 143,   4,  87,  11,  14,
         88, 393, 243,  62, 105,  47,  40,   3,  62,  33, 520, 144, 316, 364,
         83,  12, 323,  45, 235,  66, 169,   7, 357, 197, 171, 251,  21,  66,
        346, 398, 671, 130, 105, 508,  93,  14,  14,  83,  31, 112, 333, 185,
        434, 360, 123, 112, 318, 497,  65, 245,  69, 147,  28, 145,  36,  59,
         55, 164])

In [42]:
drivers = torch.cat((idle_starttime.unsqueeze(1), idle_duration.unsqueeze(1), driver_positions), 1)

### test py code

In [3]:
T0_pricing_params = (5, 0.78, 1.82)
lr = 0.01
simulation_week1 = WeeklySimulation(lr, T0_pricing_params)

In [4]:
drivers_week1_S0 = simulation_week1.simulate_drivers()

In [5]:
drivers_week1_S0.size()

torch.Size([100, 5])

In [6]:
drivers_week1_S0[0]

tensor([1.2600e+03, 9.8000e+01, 9.8811e-01, 1.5567e-01, 0.0000e+00])

In [8]:
simulation_week1.get_subblock_index(drivers_week1_S0[:, 2], drivers_week1_S0[:, 3])

tensor([14,  2,  9,  1, 13, 10,  8, 13,  7,  0,  5,  8,  3,  1,  7,  3,  6,  3,
         7,  1,  8,  2,  9, 11,  5,  7, 13, 11,  4,  6,  4,  0,  6,  7, 11,  1,
        12, 12, 14, 15,  0, 14,  3, 13,  9,  1, 13,  4,  0, 13,  1, 12, 15,  6,
         0,  2,  6,  0,  2,  8,  2, 13, 15,  8,  5, 12, 11,  8,  1,  0,  4, 11,
        11,  3, 12,  9, 15,  6,  0,  3,  0,  1,  8,  4,  2,  8,  9,  9, 12,  6,
         6, 11,  3,  3,  9,  2,  9, 12,  0,  7], dtype=torch.int32)

### TODO - 
### since in each request has to be matched to an idle driver within the same sub-block, maybe we don't need to update the driver's location after finishing the trip(to avoid loops).
### the problem is for those trips that cannot be finished within each 30-min interval


### matching - for every 30-min interval, iterate through each sub-block, for rider requests mask on the timestamp and the sub-block id; for drivers, mask on idle(not in busy_drivers and current timestamp within [idle_start_time, idle_start_time+idle_duration]) and sub-block id, randomly pick a valid driver(if no valid driver, cancel the request right away), do the match(acceptance prob. for both parties) and add the current driver and the trip end timestamp to the busy_drivers hashmap.




### if there are so many requests not able to find even a valid driver, may consider add more total drivers.

### how M0 impacts S1

### may still sample idle position and idle time every day and match every 30mins.
### a driver's reject may change the driver's mean_idle_time(for exp var to sample idle time), may or may not affect driver's initial position for the next day

### take vectors of riders and drivers and get their location's sub-block ids

In [7]:
a_r = 1.0  # Rider acceptance probability parameter
b_r = -0.1  # Rider acceptance probability parameter
a_d = 1.0  # Driver acceptance probability parameter
b_d = -0.05  # Driver acceptance probability parameter
pricing_params = T0_pricing_params

In [50]:
num_riders = 10

In [51]:
commuter_percentage, party_goer_percentage, sporadic_rider_percentage = 0.25, 0.25, 0.25

In [4]:
drivers_week1_d0 = simulation_week1.simulate_supply()

In [5]:
requests_week1_d0 = simulation_week1.simulate_demand()

In [6]:
drivers_week1_d0.size(), requests_week1_d0.size()

(torch.Size([100, 6]), torch.Size([1101, 8]))

In [7]:
#(num_drivers) * (idle_start_time_timestamps, idle_duration, idle_start_x, idle_start_y, driver_idx, idle_start_subblock_id)
drivers_week1_d0[0]

tensor([1.0800e+03, 5.6300e+02, 5.3787e-01, 3.8972e-01, 0.0000e+00, 9.0000e+00])

In [8]:
#(num_requests) * (request_timestamps, req_start_x, req_start_y, req_end_x, req_end_y, rider_idx, req_start_subblock_id, req_end_subblock_id)
requests_week1_d0[0]

tensor([4.2997e+02, 6.7464e-01, 6.8649e-01, 6.3967e-01, 4.8825e-02, 1.0000e+00,
        1.0000e+01, 8.0000e+00])

### Concerns in the ride:
### - a rider could have two rides during the morning rush hours(not sure if it makes sense), easier for vector initilization

### Concerns in matching:
### - need to iterate through every rider in the given time interval and the sub-block is both rider and driver are valid

### - to update a driver's location and idle status need to do indexing on the drivers vector and drop every driver who is on a trip when iterating through the requests(but num_drivers is quite small)

### - often times, for a given time interval and sub-block, there are much less valid drivers the requests.

### after we set the driver to busy for a matched trip, how do we reset the driver's status back to 0(need an extra variable for the trip finishing timestamp), but the problem is that since we match every 30mins, what happens in between we couldn't capture. (could change idle_start_time and idle_duration accordingly)

### if a driver start at sub-block 0 during t=0 then finish a trip and now live in sub-block 10 during t=10, it will be a valid driver in only at sub-block 0 since we do the matching every 30mins(could have a shorter time interval but will increase runtime)


### to update num_rejects for a rider, a rider could have requests in the same interval for different sub-blocks (shouldn't be a big concern as we need only a vector of (num_riders)*1)

In [18]:
#num_squares = 4
# for square_index in range(num_squares ** 2): 

match_interval_time = 30
for interval_idx, match_interval in enumerate(range(0, 24*60-1, match_interval_time)):
    for square_index in range(1):
        #replace idle_status_mask with checking on the idle_time
        #idle_status_mask = drivers_week1_d0[:, 6]==0
        idle_time_mask_left = drivers_week1_d0[:, 0]<(interval_idx+1)*match_interval_time
        idle_time_mask_right = interval_idx*match_interval_time<=drivers_week1_d0[:, 0]+drivers_week1_d0[:, 1]
        idle_location_mask = drivers_week1_d0[:, 5]==square_index

        drivers_subblock = drivers_week1_d0[idle_time_mask_left & idle_time_mask_right & idle_location_mask]


        # if len(drivers_subblock)==0:
        #     #print(f'no idle driver in this sub-block:{square_index}')
        #     continue
        # else:
        #     print(f'at least one idle driver in this sub-block:{square_index}')


        request_time_mask_left = interval_idx*match_interval_time<=riders_week1_d0[:, 0]
        request_time_mask_right = riders_week1_d0[:, 0]<(interval_idx+1)*match_interval_time
        request_location_mask = riders_week1_d0[:, 6]==square_index

        riders_subblock = riders_week1_d0[request_time_mask_left & request_time_mask_right \
                                         & request_location_mask]
        
        if len(riders_subblock)==0:
            #print(f'no idle driver in this sub-block:{square_index}')
            continue
        else:
            print(f'at least one rider request in this sub-block:{square_index}')

        #have to iterate through every rider in the given time interval and the sub-block is both rider and driver are valid

        for valid_request_id in range(riders_subblock.shape[0]):
            valid_driver = drivers_subblock.shape[0]
            selected_driver = drivers_subblock[torch.randint(0, valid_driver, (1,)).item()] if valid_driver>1 else drivers_subblock

            ride_minutes, ride_miles = simulation_week1.estimate_trip_distance_duration(riders_subblock[valid_request_id][1:5])
            price_of_ride = pricing_params[0] + pricing_params[1] * ride_minutes + pricing_params[2] * ride_miles


            rider_acceptance_prob = torch.sigmoid(a_r + b_r * price_of_ride)
            driver_acceptance_prob = torch.sigmoid(a_d + b_d * price_of_ride)

            # Determine if the ride is accepted by both rider and driver
            rider_acceptance_generator = np.random.rand()
            driver_acceptance_generator = np.random.rand()
            if rider_acceptance_generator < rider_acceptance_prob and driver_acceptance_generator < driver_acceptance_prob:
                        driver_idx = selected_driver[4]
                        #remove the current busy driver in drivers_subblock
                        drivers_subblock = drivers_subblock[drivers_subblock[:, 4]!=driver_idx]
                        if len(drivers_subblock)==0:
                             #no more valid drivers, exit the loop right away
                             break
                        #update driver's idle sub-block id to the trip destination
                        drivers[driver_idx][5]==riders_subblock[valid_request_id][7]

                        #update their idle_start_timestamp to request_time+trip_duration 
                        prev_idle_start_timestamp = drivers[driver_idx][0]
                        drivers[driver_idx][0] = riders_subblock[valid_request_id][0] + ride_minutes
                        #update idle_duration to the original idle_duration minus how much time has passed since the previous idle_start_timestamp to new idle_start_timestamp, (no negative values)
                        drivers[driver_idx][1] = max(0, drivers[driver_idx][1] - drivers[driver_idx][0] - prev_idle_start_timestamp)


                

            elif rider_acceptance_generator >= rider_acceptance_prob:
                #TODO - update rider_rejects vector
                pass


            elif driver_acceptance_generator >= driver_acceptance_prob:
                #TODO - update driver_rejects vector
                pass
        
    if len(riders_subblock)!=0 and len(drivers_subblock)!=0:
        #print(f'no idle driver in this sub-block:{square_index}')
        break
    

at least one rider request in this sub-block:0
at least one rider request in this sub-block:0


In [19]:
riders_subblock[0]

tensor([3.8136e+02, 1.3637e-01, 1.0591e-01, 9.4807e-01, 6.7556e-01, 4.0000e+01,
        0.0000e+00, 1.4000e+01])

In [24]:
riders_subblock.size()

torch.Size([2, 8])

In [23]:
riders_subblock

tensor([[3.8136e+02, 1.3637e-01, 1.0591e-01, 9.4807e-01, 6.7556e-01, 4.0000e+01,
         0.0000e+00, 1.4000e+01],
        [3.7952e+02, 9.3346e-02, 8.6899e-02, 6.8988e-01, 2.7311e-01, 1.8100e+02,
         0.0000e+00, 9.0000e+00]])

In [22]:
riders_subblock[riders_subblock[:, 4]!=1]

tensor([[3.8136e+02, 1.3637e-01, 1.0591e-01, 9.4807e-01, 6.7556e-01, 4.0000e+01,
         0.0000e+00, 1.4000e+01],
        [3.7952e+02, 9.3346e-02, 8.6899e-02, 6.8988e-01, 2.7311e-01, 1.8100e+02,
         0.0000e+00, 9.0000e+00]])

In [58]:
drivers_subblock[0]

tensor([4.2000e+02, 3.0200e+02, 2.4768e-01, 1.5847e-02, 0.0000e+00, 0.0000e+00])

In [63]:
valid_driver = drivers_subblock.shape[0]
selected_driver = drivers_subblock[torch.randint(0, valid_driver, (1,)).item()] if valid_driver>1 else drivers_subblock

In [64]:
selected_driver

tensor([[4.2000e+02, 3.0200e+02, 2.4768e-01, 1.5847e-02, 0.0000e+00, 0.0000e+00]])

In [16]:
riders_subblock

NameError: name 'riders_subblock' is not defined

In [63]:
alphas_commuters = torch.ones(int(num_riders * commuter_percentage * 2), ) * 1.5
betas_commuters = torch.ones(int(num_riders * commuter_percentage * 2), ) * 1.5
alphas_party_goers = torch.ones(int(num_riders * party_goer_percentage), ) * 0.75
betas_party_goers = torch.ones(int(num_riders * party_goer_percentage), ) * 0.75
alphas_sporadic = torch.ones(int(num_riders * sporadic_rider_percentage), ) * 0.75
betas_sporadic = torch.ones(int(num_riders * sporadic_rider_percentage), ) * 0.75


In [71]:
beta_riders = torch.cat([
    betas_commuters,
    betas_party_goers,
    betas_sporadic
])

In [72]:
beta_riders

tensor([1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 0.7500, 0.7500, 2.0000, 1.0000])

In [64]:
gamma_dist_commuters = torch.distributions.Gamma(alphas_commuters, betas_commuters)
gamma_dist_party_goers = torch.distributions.Gamma(alphas_party_goers, betas_party_goers)
gamma_dist_sporadic = torch.distributions.Gamma(alphas_sporadic, betas_sporadic)


In [65]:
lambda_riders = torch.cat([
    gamma_dist_commuters.sample(),
    gamma_dist_party_goers.sample(),
    gamma_dist_sporadic.sample()
])

In [61]:
lambda_riders

tensor([0.3423, 1.3083, 0.5971, 0.4102, 2.1167, 1.8451, 0.2329, 0.0061, 0.0694])

In [73]:
num_requests = torch.zeros(10, )
num_requests[0]+=1

In [74]:
num_requests

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [75]:
test_indices = torch.arange(10)
test_mask1 = test_indices < 2

In [76]:
num_requests[test_mask1]

tensor([1., 0.])

In [78]:
betas_test = torch.zeros(2, )

In [79]:
betas_test += num_requests[test_mask1]

In [81]:
betas_test

tensor([1., 0.])